In [1]:
import pandas as pd
import numpy as np

/Users/vega7unk/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import os
print(os.getcwd())

/Users/vega7unk/Documents/4th Sem DSI/ilab project/ilab-group-12-1-fall-detection/notebooks


In [3]:
pd.set_option('display.max_columns', None)

df = pd.read_pickle("../data/raw/df_resample_100ms.pkl")

In [4]:
df.head()

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label,subject_id,trial,age,height,weight,gender
timestamp,,,,,,,,,,,,,
1970-01-01 00:38:38.700,0.948777,-9.636166,0.002699,0.003818,0.016875,0.006643,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:38.800,0.948993,-9.759188,0.087482,0.020693,0.027565,0.015669,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:38.900,1.071514,-9.787465,-0.093610,0.168721,0.075747,0.037797,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:39.000,1.135679,-9.754036,0.065878,0.339488,0.078970,0.040134,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:39.100,1.126174,-9.443248,0.075006,0.594494,-0.012065,0.103557,BSC,1.0,1.0,32.0,180.0,85.0,M


In [5]:
data = df.copy()

In [6]:
data.drop(['subject_id','trial','age','height','weight','gender'], axis=1, inplace=True)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 846950 entries, 1970-01-01 00:38:38.700000 to 1970-01-01 04:49:55.600000
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   acc_x   846950 non-null  float64
 1   acc_y   846950 non-null  float64
 2   acc_z   846950 non-null  float64
 3   gyro_x  846950 non-null  float64
 4   gyro_y  846950 non-null  float64
 5   gyro_z  846950 non-null  float64
 6   label   846950 non-null  object 
dtypes: float64(6), object(1)
memory usage: 51.7+ MB


## Apply a low pass filter 

Using Butterworth filter, to further smooth data.

In [9]:
import scipy.signal as signal

def apply_low_pass_filter(data, cutoff=3, fs=10, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    df_filtered = data.copy()
    for col in ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']:
        df_filtered[col] = signal.filtfilt(b, a, data[col])
    return df_filtered

df_filtered = apply_low_pass_filter(data)


In [10]:
df_filtered.head()

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label
timestamp,,,,,,,
1970-01-01 00:38:38.700,0.948663,-9.636215,0.002786,0.003675,0.016768,0.006653,BSC
1970-01-01 00:38:38.800,0.926452,-9.752515,0.032618,0.029569,0.049813,0.018963,BSC
1970-01-01 00:38:38.900,1.086636,-9.797180,-0.025603,0.145156,0.084243,0.032240,BSC
1970-01-01 00:38:39.000,1.178256,-9.731493,0.007527,0.372861,0.020013,0.049501,BSC
1970-01-01 00:38:39.100,1.016914,-9.499515,0.145944,0.576895,0.024343,0.085384,BSC


cutoff=3: Retains meaningful signals below 3 Hz.

fs=10: Sampling frequency (10 Hz).

## Normalize Sensor Data

Scale sensor readings to ensure balanced input for CNN:

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
sensor_cols = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']
df_filtered[sensor_cols] = scaler.fit_transform(df_filtered[sensor_cols])


In [12]:
df_filtered.head()

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label
timestamp,,,,,,,
1970-01-01 00:38:38.700,0.209754,-2.331582,-0.203113,0.037081,0.031012,-0.012518,BSC
1970-01-01 00:38:38.800,0.203321,-2.349069,-0.194536,0.066225,0.079630,0.009396,BSC
1970-01-01 00:38:38.900,0.249716,-2.355785,-0.211275,0.196319,0.130287,0.033033,BSC
1970-01-01 00:38:39.000,0.276253,-2.345908,-0.201750,0.452604,0.035785,0.063763,BSC
1970-01-01 00:38:39.100,0.229522,-2.311028,-0.161954,0.682246,0.042157,0.127645,BSC


In [14]:
# Specify the directory and filename
directory = "../data"
filename = "df_filtered_cnn.pkl"
filepath = os.path.join(directory, filename)

# Create the directory if it does not exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the DataFrame as a pickle file
df_filtered.to_pickle(filepath)